In [381]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy.spatial.distance import cdist
from scipy.integrate import solve_ivp
from common_functions import coordinate_extractor, mean_squared_error, phase_portrait

ImportError: cannot import name 'phase_portrait' from 'common_functions' (C:\Users\91901\OneDrive\Desktop\Task_3\common_functions.py)

In [369]:
matrix_t0 = coordinate_extractor("linear_vectorfield_data_x0.txt")
matrix_t1 = coordinate_extractor("linear_vectorfield_data_x1.txt")

In [370]:
def rbf_differential_calculator(t, y, centers, eps, C):
    
    """
    
    Returns vector field of a single point (rbf)
    
    Args:
    
        - t (float): time (for solve_ivp)
        - y (float): single point
        - centers (np.array): all centers
        - eps (float): radius of gaussians
        - C (numpy.ndarray): coefficient matrix, found with least squares
        
    Returns:
        
        - numpy.ndarray - The derivative of phi with respect to time.
    """
    
    y = y.reshape(1, y.shape[-1])
    phi = np.exp(-cdist(y, centers) ** 2 / eps ** 2)
    return phi @ C

In [371]:
def vector_field(dt, x0, x1):
    """
    
    Estimates the vector field based on the given time step.

    Args:
        - dt (float): Time step.
        - x0 (numpy.ndarray): Initial state.
        - x1 (numpy.ndarray): Target state.

    Returns:
        - numpy.ndarray: Estimated vector field.
        
    """
    
    field = (x1-x0)/dt

    return field

In [372]:
def approx_nonlin_func(x0, v, n_bases, eps, centers):
    """
    
    Approximates a nonlinear function using python supported LSTSQ library.

    Args:
    
        - x0 (numpy.ndarray): Input data points.
        - v (numpy.ndarray): Output values.
        - n_bases (int): No. of RBF clusters
        - eps (float): Width of the basis functions.
        - centers (numpy.ndarray): Chosen centers of the basis functions.

    Returns:
    
        - sol (numpy.ndarray): Coefficient matrix.
        - residuals (numpy.ndarray): Residuals of the least squares solution.
        - eps (float): Width of the basis functions.
        - differentials (numpy.ndarray): Evaluated basis functions.

    """
   # Evaluate the basis functions on the whole data
   # cdist is used for calculating differences
    differentials = np.exp(-cdist(x0, centers)**2 / eps**2)
    sol, residuals, _, _ = np.linalg.lstsq(a=differentials, b=v, rcond=1e-5)
    
    return sol, residuals, eps, differentials

In [373]:
def trajectory(x0, x1, approx_func, args, find_best_dt, end_time, plot=False):
    """
    
    Solves the trajectory using the solve_ivp under SciPy.

    Args:
        
        - x0 (numpy.ndarray): Initial state.
        - x1 (numpy.ndarray): Target state.
        - approx_func (function): Approximation function.
        - args (list): Additional arguments for the approximation function.
        - find_best_dt (bool): Whether to find the best time step.
        - end_time (float): End time for solving the trajectory.
        - plot (bool): Whether to plot the results.

    Returns:
        
        - tuple: Predicted trajectory, best time step (if applicable), best MSE (if applicable).
        
    """
    # Variable initialization
    best_dt = -1
    best_mse = math.inf
    pred_matrix_t1 = []
    t_eval = np.linspace(0, end_time, 100)
    sols = []
    
    for i in range(len(x0)):
        sol = solve_ivp(approx_func, [0, end_time], x0[i], args=args, t_eval=t_eval)  
        pred_matrix_t1.append([sol.y[0, -1], sol.y[1, -1]])  # storing final solutions
        
        if find_best_dt:
            # to find best dt then all the different snapshots in time have to be saved
            sols.append(sol.y)
        # plot the trajectory (orange) and ground truth end point (blue)
        
        if plot:
            plt.scatter(x1[i, 0], x1[i, 1], c='blue', s=10)
            plt.scatter(sol.y[0, :], sol.y[1, :], c='orange', s=4)
            
    if find_best_dt:
        # try all the different moments in time, check if it is the best time
        for i in range(len(t_eval)):
            pred = [[sols[j][0][i], sols[j][1][i]] for j in range(len(sols))]
            mse = np.mean(np.linalg.norm(pred - x1, axis=1)**2)
            # if mse found is best yet, update the variables
            if mse < best_mse:
                best_mse = mse
                best_dt = t_eval[i]
    if plot:
        plt.figure(figsize=(14,14))
        plt.show()
    return x1_pred, best_dt, best_mse


In [374]:
def optimal_rbf_configuration(x0, x1, dt=0.1, end_time=0.5):
    """
    
    Finds the best configuration for Radial Basis Function (RBF) approximation.

    Args:
        
        - x0 (numpy.ndarray): Initial state.
        - x1 (numpy.ndarray): Final state.
        - dt (float): Time step for trajectory integration (default: 0.1).
        - end_time (float): End time for trajectory integration (default: 0.5).

    Returns:
        
        - Tuple: (final_best_mse, final_best_eps, final_best_n_bases, final_best_dt, final_centers)
        
    """
    
    final_best_mse, final_best_eps, final_best_n_bases, final_best_dt = math.inf, -1, -1, -1  # initialize variables
    n_bases_trials = [int(i) for i in np.linspace(100, 1001, 20)]  # define search space for n_bases
    for n_bases in n_bases_trials:
        centers = x0[np.random.choice(range(x0.shape[0]), size=n_bases)] 
        for eps in (0.3, 0.5, 0.7, 1.0, 5.0, 10.0, 20.0):
            v = vector_field(dt, x0, x1)  # estimate vector field
            C, res, eps, phi = approx_nonlin_func(x0, v, n_bases=n_bases, eps=eps, centers=centers)
            x1_pred, best_dt, best_mse = trajectory(x0, x1, rbf_differential_calculator, find_best_dt=True, args=[centers, eps, C], end_time=end_time, plot=False)
            if final_best_mse > best_mse:  # if new mse is better then update all return variables
                final_best_mse, final_best_eps, final_best_n_bases, final_best_dt, final_centers  = best_mse, eps, n_bases, best_dt, centers
    print(f"Best configuration: eps = {final_best_eps}, n_bases = {final_best_n_bases}, and dt = {final_best_dt} gives MSE = {final_best_mse}")
    return final_best_mse, final_best_eps, final_best_n_bases,final_best_dt,final_centers

In [375]:
 # Step 1: Read vector field data
x0 = matrix_t0
x1 = matrix_t1


# Step 2: Find the best RBF configuration
final_best_mse, eps, n_bases, final_best_dt, centers = optimal_rbf_configuration(x0, x1)

# Step 3: Estimate vector field and approximate non-linear function
dt = 0.1
v = vector_field(dt, x0, x1)
C, res, eps, phi = approx_nonlin_func(x0, v, n_bases=n_bases, eps=eps, centers=centers)

# Step 4: Solve trajectory using the best configuration and plot results
end_time = final_best_dt
x1_pred, _, _ = trajectory(x0, x1, rbf_differential_calculator, find_best_dt=False, end_time=end_time, args=[centers, eps, C], plot=False)

Best configuration: eps = 0.3, n_bases = 669, and dt = 0.10606060606060606 gives MSE = 2.5782151020803094e-06


In [378]:
phase_portrait(C, "Phase Portrait - Non linear System")

NameError: name 'phase_portrait' is not defined